In [2]:
import os
import torch 
import torch.nn as nn
import numpy as np
import argparse
import pickle
from torch.autograd import Variable
from packages.vocab import Vocab
from packages.batch import Batch
from models.lstm.model import RNNLM
from slimit.lexer import Lexer
from functions import detach

In [3]:
outputs = Variable(torch.Tensor(np.arange(1000,dtype=float).reshape([10,20,-1]))).cuda()

In [29]:
b = np.random.randint(100,size=(outputs.size(0),outputs.size(1)))
c = np.vstack([np.sort(row) for row in b])
d = np.array(c<50,dtype=int)
targets = d

In [30]:
(targets!=0).sum(1)

array([ 8, 12, 14, 10,  8, 12,  8,  9, 14,  7])

In [35]:
out = []
for i, length in enumerate((targets!=0).sum(1)):
    print(length)
    out.append(np.arange(0,length)+i*outputs.size(1))

8
12
14
10
8
12
8
9
14
7


In [38]:
valid = np.hstack(out)

In [49]:
Valid = torch.LongTensor(valid).cuda()

In [ ]:
d.sum(1)

In [ ]:
in_lines = list((d!=0).sum(1).cpu().data.numpy())

In [ ]:
out_list = []
for i, item in enumerate(d.sum(1)):
    out_list.extend(list(np.arange(item,dtype=int)+i*a.size(1)))
out_np = np.array(out_list,dtype=int)

In [ ]:
d.view(-1,1)[torch.LongTensor(out_np).cuda()]

In [ ]:
a.cuda().view(-1,a.size(2))[torch.LongTensor(out_np).cuda()]

In [56]:
(targets!=0).sum(1)

Variable containing:
  8
 12
 14
 10
  8
 12
  8
  9
 14
  7
[torch.cuda.ByteTensor of size 10 (GPU 0)]

In [54]:
outputs.view(-1,outputs.size(2))[Valid]

Variable containing:
    0     1     2     3     4
    5     6     7     8     9
   10    11    12    13    14
   15    16    17    18    19
   20    21    22    23    24
   25    26    27    28    29
   30    31    32    33    34
   35    36    37    38    39
  100   101   102   103   104
  105   106   107   108   109
  110   111   112   113   114
  115   116   117   118   119
  120   121   122   123   124
  125   126   127   128   129
  130   131   132   133   134
  135   136   137   138   139
  140   141   142   143   144
  145   146   147   148   149
  150   151   152   153   154
  155   156   157   158   159
  200   201   202   203   204
  205   206   207   208   209
  210   211   212   213   214
  215   216   217   218   219
  220   221   222   223   224
  225   226   227   228   229
  230   231   232   233   234
  235   236   237   238   239
  240   241   242   243   244
  245   246   247   248   249
  250   251   252   253   254
  255   256   257   258   259
  260   261   262  

In [ ]:
help(np.random.randint)

In [ ]:
# arguments related to the dataset
parser = argparse.ArgumentParser()

# about running mode
parser.add_argument("--mode",type=str, help='whether "train" or "test" data')
parser.add_argument("--dataset_name",type=str, help='name of preprocessed dataset')
parser.add_argument("--dataset_dir",type=str, default= '/home/irteam/users/data/150kJavaScript/', 
                    help='name of preprocessed dataset')
parser.add_argument("--startfrom",type=int, default=0, help='whether starting from a previous model')


# model information
parser.add_argument("--embed_size",type=int, default=128, help='embedding size')
parser.add_argument("--hidden_size",type=int, default=1024, help='hidden size')
parser.add_argument("--num_layers",type=int, default=1, help='number of LSTM layers')
parser.add_argument("--num_epochs",type=int, default=10, help='number of epochs to run')
parser.add_argument("--batch_size",type=int, default=100, help='size of minibatch')
parser.add_argument("--seq_length",type=int, default=50, help='length of sequence size')
parser.add_argument("--lr",type=float, default=0.002, help='learning rate size')
parser.add_argument("--vocab_size",type=int, default=50000, help='vocab_size')

# args = parser.parse_args()
# with open(os.path.join(args.dataset_dir,args.dataset_name,'list_of_files.txt')) as f:
#     file_list = f.read().split('\n')

# batch = Batch(file_dir=args.dataset_dir,file_list=file_list,batch_size=args.batch_size,
#               in_seq=args.seq_length,out_seq=args.seq_length)

# idx2id = np.load('vocab/data_lexed/idx2id.npy').item()
# idx2tok = np.load('vocab/data_lexed/idx2tok.npy').item()
# idx2reg = np.load('vocab/data_lexed/idx2reg.npy').item()
# word_list = [x for (_,x) in idx2tok.items()] + [x for (_,x) in idx2reg.items()][:1000] + \
#         [x for (_,x) in idx2id.items()][:args.vocab_size]
# vocab = Vocab(args.vocab_size)
# vocab.add_to_vocab(word_list)

# import lexer
lexer = Lexer()

# load model
if args.startfrom==0:    
    model = RNNLM(vocab.max_size, embed_size=args.embed_size, hidden_size=args.hidden_size,
                 num_layers = args.num_layers)
else:
    model = torch.load(os.path.join(args.dataset_dir)) # To-do
model.cuda()

# set loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

######################### training ###########################
step = 99999
if args.mode=='train':
    for epoch in range(args.num_epochs):
        batch.next_epoch(args.batch_size) # initialize batch data
        batch.initialize_states(args.num_layers, args.hidden_size)
        total_steps=step
        step=0
        while(batch.epoch_end==0):
            step+=1
            # update the minibatch inputs / outputs
            model.zero_grad()
            batch.get_minibatch(0)
            inputs_np = np.array([vocab.word_list_to_idx_list(line) for
                                 line in batch.batch_in],dtype=int)
            targets_np = np.array([vocab.word_list_to_idx_list(line) for
                                 line in batch.batch_out],dtype=int)
            inputs = Variable(torch.LongTensor(inputs_np)).cuda()
            targets = Variable(torch.LongTensor(targets_np)).cuda()

            outputs, states = model(inputs,batch.states)
            batch.states = detach(states)
            loss = criterion(outputs, targets.view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm(model.parameters(),0.5)
            optimizer.step()

            batch.next_minibatch()
            if step%100==0:
                print ('Epoch [%d/%d], Loss: %.3f, Steps: [%d/%d], Perplexity: %5.2f' %
               (epoch+1, num_epochs, loss.data[0], step, total_steps, np.exp(loss.data[0])))
        torch.save(f='models/rnn_lm_types_3_epoch_%d.pckl'%(epoch+1),obj=model)        

In [ ]:
# Truncated Backpropagation 
def detach(states):
    return [state.detach() for state in states] 

In [ ]:
# Training
steps=99999
for epoch in range(num_epochs):
#     states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
#               Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())
    batch.next_epoch(batch_size)
    batch.initialize_states(num_layers, hidden_size)
    total_steps=steps
    step=0
    while(batch.epoch_end==0):
        step+=1
        # update the minibatch inputs / outputs
        model.zero_grad()
        batch.get_minibatch(0)
        inputs_np = np.array([vocab.word_list_to_idx_list(line) for
                             line in batch.batch_in],dtype=int)
        targets_np = np.array([vocab.word_list_to_idx_list(line) for
                             line in batch.batch_out],dtype=int)
        inputs = Variable(torch.LongTensor(inputs_np)).cuda()
        targets = Variable(torch.LongTensor(targets_np)).cuda()
        
        outputs, states = model(inputs,batch.states)
        batch.states = detach(states)
        loss = criterion(outputs, targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(),0.5)
        optimizer.step()
        
        batch.next_minibatch()
        if step%100==0:
            print ('Epoch [%d/%d], Loss: %.3f, Steps: [%d/%d], Perplexity: %5.2f' %
           (epoch+1, num_epochs, loss.data[0], step, total_steps, np.exp(loss.data[0])))
    torch.save(f='models/rnn_lm_types_2_epoch_%d.pckl'%(epoch+1),obj=model)

In [ ]:
#     with open(train_path+'list_of_files.txt') as f:
#         input_files = f.readlines()
#     i = 0
#     for file in input_files:
#         i+=1
#         with open(train_path+file.strip()) as f:
#             lines = f.readlines()
#         out_lines = []
#         for line in lines:
#             line = line.strip()
#             out_lines.append(vocab.word_list_to_idx_list(line.split())[:50])
#         batch_size = len(lines)
#         ids = torch.LongTensor(np.array(out_lines[:batch_size]))
#         states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
#                   Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())

# #     for i in range(0, ids.size(1) - seq_length, seq_length):
# #         # Get batch inputs and targets
#         inputs = Variable(ids[:, :seq_length-1]).cuda()
#         targets = Variable(ids[:, 1:seq_length].contiguous()).cuda()
        
#         # Forward + Backward + Optimize
#         model.zero_grad()
#         states = detach(states)
#         outputs, states = model(inputs, states) 
#         loss = criterion(outputs, targets.view(-1))
#         loss.backward()
#         torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
#         optimizer.step()

In [ ]:
states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
          Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())

In [ ]:
loss.data[0]

In [ ]:
type(S)

In [ ]:
H2 = torch.chunk(H,batch.batch_size,dim=1)

In [ ]:
H2[0]

In [ ]:
# Sampling
sample_path = '/home/irteam/users/data/150kJavaScript/samples.txt'
with open(sample_path, 'w') as f:
    # Set intial hidden ane memory states
    state = (Variable(torch.zeros(num_layers, 1, hidden_size)).cuda(),
         Variable(torch.zeros(num_layers, 1, hidden_size)).cuda())

    # Select one word id randomly
    prob = torch.ones(vocab_size)
    input = Variable(torch.multinomial(prob, num_samples=1).unsqueeze(1),
                     volatile=True).cuda()

    for i in range(num_samples):
        # Forward propagate rnn 
        output, state = model(input, state)
        
        # Sample a word id
        prob = output.squeeze().data.exp().cpu()
        word_id = torch.multinomial(prob, 1)[0]
        
        # Feed sampled word id to next time step
        input.data.fill_(word_id)
        
        # File write
        word = corpus.dictionary.idx2word[word_id]
        word = '\n' if word == '<eos>' else word + ' '
        f.write(word)

        if (i+1) % 100 == 0:
            print('Sampled [%d/%d] words and save to %s'%(i+1, num_samples, sample_path))

In [ ]:
# Testing
for epoch in range(1):
#     states = (Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda(),
#               Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda())
    batch.next_epoch(batch_size)
    batch.initialize_states(num_layers, hidden_size)
    step=0
    while(batch.epoch_end==0):
        step+=1
        # update the minibatch inputs / outputs
        batch.get_minibatch(0)
        inputs_np = np.array([vocab.word_list_to_idx_list(line) for
                             line in batch.batch_in],dtype=int)
        targets_np = np.array([vocab.word_list_to_idx_list(line) for
                             line in batch.batch_in],dtype=int)
        inputs = Variable(torch.LongTensor(inputs_np)).cuda()
        targets = Variable(torch.LongTensor(targets_np)).cuda()
        
        outputs, states = model(inputs,batch.states)
        batch.states = detach(states)
        break

In [ ]:
out = outputs.view(batch_size,seq_length,-1).max(2)[1]

In [ ]:
input

In [ ]:
for i in range(len(targets)):
    line0 = inputs[i]
    line1 = targets[i]
    line2 = out[i]
    print(''.join(vocab.idx_list_to_word_list(line0.cpu().data.numpy())))    
    print(''.join(vocab.idx_list_to_word_list(line1.cpu().data.numpy())))
    print(''.join(vocab.idx_list_to_word_list(line2.cpu().data.numpy())))
    print('\n\n')

In [ ]:
inputs_np = ids.cpu().numpy()
targets_np = targets.cpu().data.numpy()
predicted_np = predicted_outs.cpu().data.numpy()

In [ ]:
for i in range(targets.size(0)):
    print(vocab.idx_list_to_word_list(inputs_np[i]))
    print(vocab.idx_list_to_word_list(predicted_np[i]))
    print(vocab.idx_list_to_word_list(targets_np[i]))
    print('\n')